In [2]:
import pandas as pd

In [3]:
X = pd.read_csv("../data/forecasts_for_ml/X.csv")
y = pd.read_csv("../data/forecasts_for_ml/y.csv")

In [5]:
X.columns

Index(['Unnamed: 0', 'dayofweek', 'quarter', 'month', 'year', 'weekofyear',
       'horizon', 'hurst', 'unitroot_pp', 'unitroot_kpss', 'stability',
       'nperiods', 'seasonal_period', 'trend', 'spike', 'linearity',
       'curvature', 'e_acf1', 'e_acf10', 'x_pacf5', 'diff1x_pacf5',
       'diff2x_pacf5', 'nonlinearity', 'lumpiness', 'alpha', 'beta',
       'flat_spots', 'entropy', 'crossing_points', 'arch_lm', 'x_acf1',
       'x_acf10', 'diff1_acf1', 'diff1_acf10', 'diff2_acf1', 'diff2_acf10',
       'autoreg'],
      dtype='object')